In [1]:
import pandas as pd
from pandas import read_csv
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier

# start with linear classifiers, non-linear ones and eventually the more complex neural nets

In [2]:
# load a single file as a numpy array
# def load_file(filepath):
#     dataframe = read_csv(filepath, header=None, delim_whitespace=True)
#     return dataframe.values

# load a dataset group, such as train or test
# def load_dataset_group(group, prefix=''):
#     # load input data
#     X = load_file(prefix + group + '/X_'+group+'.txt')
#     # load class output
#     y = load_file(prefix + group + '/y_'+group+'.txt')
#     return X, y

# load the dataset, returns train and test X and y elements
# def load_dataset(prefix=''):
#     # load all train
#     trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

# def load_dataset(prefix=''):
#     # load all train
#     train = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

In [3]:
# path = '../data/processed/flori5hz.csv'
# values = load_file(path)
# #X, y = load_dataset_group()
# values

In [99]:
train_df = pd.read_csv('../data/processed/train.csv')

test_df = pd.read_csv('../data/processed/test.csv')

In [100]:
X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

In [115]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
    # nonlinear models
    #models['knn'] = KNeighborsClassifier()
    models['cart'] = { 'clf': DecisionTreeClassifier(), 'params': {}}
    #models['svm'] = SVC()
    #models['bayes'] = GaussianNB()
    # ensemble models
#     models['bag'] = BaggingClassifier(n_estimators=100)
#     models['rf'] = RandomForestClassifier(n_estimators=100)
#     models['et'] = ExtraTreesClassifier(n_estimators=100)
#     models['gbm'] = GradientBoostingClassifier(n_estimators=100)
    #models['bag'] = BaggingClassifier()
    models['rf'] = { 'clf': RandomForestClassifier(n_jobs=-1), 'params': {'n_estimators': [100, 200],
#                                                                            'bootstrap': [False],
#                                                                            'min_samples_split': [10, 20, 40, 50],
#                                                                            'max_features': ['log2', 'sqrt','auto', None], 
                                                                           'criterion': ['entropy', 'gini']}}
    #models['et'] = ExtraTreesClassifier()
    #models['gbm'] = GradientBoostingClassifier()
    # sgd is sensitive to feature scaling
    #models['sgd'] = SGDClassifier()
    #models['gp'] = GaussianProcessClassifier()
    #models['mlp'] = MLPClassifier()
    print('Defined %d models' % len(models))
    return models

In [116]:
# get model list
models = define_models()

Defined 2 models


In [117]:
# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
    # fit the model
    model.fit(trainX, trainy)
    # make predictions
    yhat = model.predict(testX)
    # evaluate predictions
    accuracy = balanced_accuracy_score(testy, yhat)
    #roc_auc = roc_auc_score(testy, yhat)
    return accuracy * 100.0
    #return roc_auc * 100.0

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
    results = dict()
    for name, model in models.items():
        # evaluate the model
        results[name] = evaluate_model(trainX, trainy, testX, testy, model['clf'])
        # show process
        print('>%s: %.3f' % (name, results[name]))
    return results

# print and plot the results
def summarize_results(results, maximize=True):
    # create a list of (name, mean(scores)) tuples
    mean_scores = [(k,v) for k,v in results.items()]
    # sort tuples by mean score
    mean_scores = sorted(mean_scores, key=lambda x: x[1])
    # reverse for descending order (e.g. for accuracy)
    if maximize:
        mean_scores = list(reversed(mean_scores))
    print()
    for name, score in mean_scores:
        print('Name=%s, Score=%.3f' % (name, score))

In [104]:
# evaluate models
results = evaluate_models(X_train, y_train, X_test, y_test, models)

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


>rf: 68.030


In [105]:
### summarize results
summarize_results(results)


Name=rf, Score=68.030


In [121]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from pandas import DataFrame
import numpy as np

all_df = train_df.append(test_df)
all_df.reset_index(drop=True, inplace=True)

X_all = all_df.drop(['state', 'name'], axis=1)
y_all = all_df['state']

groups = all_df['name']

scorer = make_scorer(balanced_accuracy_score)

outcomes = []

def run_nested_logo(clf, params, scorer):
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_all, groups=groups):
        group += 1
        X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
        y_train, y_test = y_all.iloc[train_index], y_all.iloc[test_index]
        
        inner_groups = all_df.iloc[all_df.index.isin(train_index)]['name']

        logo = LeaveOneGroupOut()
        grid_obj = GridSearchCV(clf, params, scoring=scorer, cv=logo.split(X_train, groups=inner_groups), n_jobs=-1)
        grid_obj = grid_obj.fit(X_train, y_train)
        
        print(grid_obj.best_estimator_)
        clf = grid_obj.best_estimator_

        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        accuracy = balanced_accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome))
    
def evaluate_tuned_models(models, scorer):
    results = dict()
    for name, model in models.items():
        print('>%s' % (name))
        clf, params = model['clf'], model['params']
        run_nested_logo(clf, params, scorer)

evaluate_tuned_models(models, scorer)


>cart


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Group 1 accuracy: 0.4192070445199356


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Group 2 accuracy: 0.3643525071638253


...

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Group 2 accuracy: 0.36264726271324144


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Group 3 accuracy: 0.6037061981238558


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Group 4 accuracy: 0.6668104191823357
Mean Accuracy: 0.5061984668146853


In [21]:
# Random Forest

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(X_train, y_train)
# y_pred = random_forest.predict(X_test)
# y_pred
# acc_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)

# cnt = 0
# sitting = 0
# standing = 0
# for i in range(0, len(y_pred)):
#     if not y_pred[i] == y_test[i]:
#         cnt += 1
#         if y_pred[i] == 1:
#             sitting += 1
#         else:
#             standing += 1
#         print('predicted', y_pred[i], 'actual', y_test[i])
        
# print('\nTotal:', len(y_test))
# print('Wrong:', cnt)
# print('Accuracy:', acc_random_forest)
# print('False positive:', sitting)
# print('False negative:', standing)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

groups = train_df['name']



logo = LeaveOneGroupOut()

# Choose the type of classifier. 
clf = RandomForestClassifier(n_jobs=-1)

depths = [2, 4, 8, 16, 32, 64, 80, 100]

# Choose some parameter combinations to try
parameters = {'n_estimators': [100, 200],
              'bootstrap': [False],
              'min_samples_split': [10, 20, 40, 50],
               'max_features': ['log2', 'sqrt','auto', None], 
               'criterion': ['entropy', 'gini']
#               'max_depth': depths
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(balanced_accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer, cv=logo.split(X_train, groups=groups), n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print(balanced_accuracy_score(y_test, predictions))

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.7017667184175723


In [53]:
grid_obj.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'min_samples_split': 10,
 'n_estimators': 200,
 'warm_start': False}

In [43]:
grid_obj.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [164]:
from sklearn.model_selection import LeaveOneGroupOut
from pandas import DataFrame
import numpy as np

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

X_all = X_train.append(X_test)
y_all = y_train.append(y_test)

groups = train_df.append(test_df)['name']



outcomes = []

# clf = RandomForestClassifier(criterion='gini',
#                              max_features=None,
#                              min_samples_split=5,
#                              n_estimators=100)

def run_logo(clf):
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_all, groups=groups):
        group += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = balanced_accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_logo(clf)


Group 1 accuracy: 0.5316714484053592
Group 2 accuracy: 0.3710382820929218
Group 3 accuracy: 0.600285689702276
Group 4 accuracy: 0.7082916451037894
Mean Accuracy: 0.5528217663260866


In [4]:
# testing keras neural network models
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

In [30]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    print('loaded', len(loaded[0]))
    loaded = dstack(loaded)
    print('stacked', loaded.shape)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, freq, win, prefix=''):
    filepath = prefix + group + '/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body acceleration
#     filenames += ['body_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body gyroscope
#     filenames += ['body_gyro_x_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_y_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_z_'+group+'_'+win+'_'+freq+'.csv']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/state_'+group+'_'+win+'_'+freq+'.csv')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(freq, win, prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', freq, win, prefix)
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', freq, win, prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    print(testy)
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(freq, win, repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset(freq=freq, win=win, prefix='../data/processed/')
    score = evaluate_model(trainX, trainy, testX, testy)
    print(score)
    # repeat experiment
#     scores = list()
#     for r in range(repeats):
#         score = evaluate_model(trainX, trainy, testX, testy)
#         score = score * 100.0
#         print('>#%d: %.3f' % (r+1, score))
#         scores.append(score)
#     # summarize results
#     summarize_results(scores)

# run the experiment
freq = '50'
win = '50'
run_experiment(freq, win)

loaded 1160
stacked (1160, 50, 3)
(1160, 50, 3) (1160, 1)
loaded 225
stacked (225, 50, 3)
(225, 50, 3) (225, 1)
[[ 0]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [-1]
 [-1]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [-1]
 [-1]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [-1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 1]
 [-1]
 [-1]
 [-1]
 [ 0]
 [ 0]
 [ 0]
